#Trabalho de Probabilidade T1 Classificador Bayesiano Simples

 Importa a bilbioteca pandas, lê o arquivo .CSV da nome as colunas dele , após isso limpa os dados e faz uma verificação via impressão dos dados
 

In [15]:
import pandas as pd

df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")

df.columns = ["idade", "classe-de-trabalho", "peso-final", "educação",
              "anos-de-educação", "estado-civil", "ocupação",
              "relacionamento", "raça", "sexo", "ganho-de-capital",
              "perda-de-capital", "horas-por-semana", "país-de-origem",
              "renda"]


for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].str.strip()
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   idade               32560 non-null  int64 
 1   classe-de-trabalho  32560 non-null  object
 2   peso-final          32560 non-null  int64 
 3   educação            32560 non-null  object
 4   anos-de-educação    32560 non-null  int64 
 5   estado-civil        32560 non-null  object
 6   ocupação            32560 non-null  object
 7   relacionamento      32560 non-null  object
 8   raça                32560 non-null  object
 9   sexo                32560 non-null  object
 10  ganho-de-capital    32560 non-null  int64 
 11  perda-de-capital    32560 non-null  int64 
 12  horas-por-semana    32560 non-null  int64 
 13  país-de-origem      32560 non-null  object
 14  renda               32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Calcula tamanho total do DataFrame

In [16]:
EspacoAmostral = len(df)
print(EspacoAmostral)

32560


#TREINAMENTO
Divide o dataframe em 80/20, usando uma seed para salvar os casos aleatórios

In [17]:
from sklearn.model_selection import train_test_split

seed = 8
test_size = 0.2
df_train, df_test = train_test_split(df, test_size=test_size, random_state=seed)


##Explicação das funções

**__init__**(self): Inicializa os atributos priori e condicionais, que armazenam as probabilidades a priori e condicionais de cada classe, respectivamente.

**calcular_priori**(self, y): Calcula as probabilidades a priori de cada classe com base nos rótulos de classe fornecidos, contando o número de ocorrências de cada classe e calculando a proporção em relação ao número total de exemplos.

**calcular_cond**(self, X, y): Calcula as probabilidades condicionais das características para cada classe, iterando sobre as características do conjunto de dados de treinamento, contando o número de ocorrências de cada valor de característica para cada classe e calculando a proporção em relação ao número total de exemplos dessa classe.

**prever**(self, X): Realiza a previsão da classe para cada exemplo no conjunto de dados de teste, calculando as probabilidades de pertencer a cada classe com base nas probabilidades a priori e condicionais, e selecionando a classe com a maior probabilidade como a previsão para cada exemplo.

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import numpy as np
class ClassificadorBayesianoSimples:
    def __init__(self):
        self.priori = {}  # Probabilidades a priori de cada classe
        self.condicionais = {}  # Probabilidades condicionais das características para cada classe
    
    def calcular_priori(self, y):
        classes, contagens = np.unique(y, return_counts=True)
        total_exemplos = len(y)
        for classe, contagem in zip(classes, contagens):
            self.priori[classe] = contagem / total_exemplos
    
    def calcular_cond(self, X, y):
        for classe in self.priori:
            exemplos_classe = X[y == classe]
            self.condicionais[classe] = {}
            for caracteristica in range(X.shape[1]):
                valores, contagens = np.unique(exemplos_classe[:, caracteristica], return_counts=True)
                total_exemplos_classe = len(exemplos_classe)
                self.condicionais[classe][caracteristica] = dict(zip(valores, contagens / total_exemplos_classe))
    
    def prever(self, X):
        previsoes = []
        for exemplo in X:
            probabilidades = {}
            for classe in self.priori:
                probabilidade = np.log(self.priori[classe])
                for caracteristica, valor in enumerate(exemplo):
                    if valor in self.condicionais[classe][caracteristica]:
                        probabilidade += np.log(self.condicionais[classe][caracteristica][valor])
                probabilidades[classe] = probabilidade
            previsoes.append(max(probabilidades, key=probabilidades.get))
        return previsoes

Retira o coluna "renda" e converte em matriz(usando numpy).
Depois cria o modelo e calcula as probabilidades a priori das classes com base nos rotulos do conjunto da base de treino. Após isso são realizados os calculos no dataframe teste. Como ultimo passo usamos a função importada da sklearn para calcular a acuracia do modelo, que com base na seed deve ser próximo dos 63%. Lembrando que o valor deve mudar quando alteramos a seed.

In [19]:
# Preparar dados para treinamento e teste
X_train = df_train.drop("renda", axis=1).values
y_train = df_train["renda"].values

X_test = df_test.drop("renda", axis=1).values
y_test = df_test["renda"].values

# Treinar modelo bayesiano simples
modelo = ClassificadorBayesianoSimples()
modelo.calcular_priori(y_train)
modelo.calcular_cond(X_train, y_train)

# Fazer previsões no conjunto de teste
previsoes = modelo.prever(X_test)

# Calcular acurácia do modelo
acuracia = accuracy_score(y_test, previsoes)
print("Acurácia do modelo:", acuracia*100)

Acurácia do modelo: 62.92997542997543


A acurácia calculada foi de 62.92997542997543
#END
